In [1]:
!pip install phe

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.7/53.7 kB 579.6 kB/s eta 0:00:00


In [2]:
import numpy as np
import pandas as pd
from sklearn.datasets import make_classification
from scipy.special import expit
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from phe import paillier
import time
import psutil
import os
from threading import Thread, Lock

In [3]:
# Load datasets
X_train = pd.read_csv('X_train3.csv').to_numpy()
X_test = pd.read_csv('X_test3.csv').to_numpy()
y_train = pd.read_csv('y_train3.csv').to_numpy().reshape(-1)
y_test = pd.read_csv('y_test3.csv').to_numpy().reshape(-1)

In [4]:
# Paillier key generation
public_key, private_key = paillier.generate_paillier_keypair()

In [5]:
# Scaling
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [6]:
# Lock for controlling CPU monitoring
monitor_lock = Lock()

In [7]:
# Function to handle encrypted multiplication at the edge device
def paillier_multiplication_edge(a, b):
    with monitor_lock:  # Acquire lock to exclude this part from CPU and RAM monitoring
        a = private_key.decrypt(a)
        a = round(a, 7)
        a = a * b
    return a

In [8]:
# Function to handle encrypted multiplication at the cloud
def paillier_multiplication_cloud(a, b):
    noise1 = 1
    noise2 = 2
    a = a + noise1
    sub_mass = (b * noise1) * (-1)
    sol = paillier_multiplication_edge(a, b)
    sol = sol + sub_mass
    return sol

In [9]:

# Function to apply the sigmoid function on encrypted data
def sigmoid(z):
    with monitor_lock:  # Acquire lock to exclude this part from CPU and RAM monitoring
        z = private_key.decrypt(z)
        z = expit(z)
        z = public_key.encrypt(z)
    return z

In [10]:
m, n = X_train.shape
weights = np.zeros(n)
bias = 0
learning_rate = 0.01

In [11]:
row=1
enc_X_train=[]
for i in X_train:
  row_x=[]
  for j in i:
    k=public_key.encrypt(j)
    row_x.append(k)
  enc_X_train.append(row_x)
  print(f"row {row} encrypted")
  row=row+1
enc_y_train = [public_key.encrypt(int(i)) for i in y_train]

row 1 encrypted
row 2 encrypted
row 3 encrypted
row 4 encrypted
row 5 encrypted
row 6 encrypted
row 7 encrypted
row 8 encrypted
row 9 encrypted
row 10 encrypted
row 11 encrypted
row 12 encrypted
row 13 encrypted
row 14 encrypted
row 15 encrypted
row 16 encrypted
row 17 encrypted
row 18 encrypted
row 19 encrypted
row 20 encrypted
row 21 encrypted
row 22 encrypted
row 23 encrypted
row 24 encrypted
row 25 encrypted
row 26 encrypted
row 27 encrypted
row 28 encrypted
row 29 encrypted
row 30 encrypted
row 31 encrypted
row 32 encrypted
row 33 encrypted
row 34 encrypted
row 35 encrypted
row 36 encrypted
row 37 encrypted
row 38 encrypted
row 39 encrypted
row 40 encrypted
row 41 encrypted
row 42 encrypted
row 43 encrypted
row 44 encrypted
row 45 encrypted
row 46 encrypted
row 47 encrypted
row 48 encrypted
row 49 encrypted
row 50 encrypted
row 51 encrypted
row 52 encrypted
row 53 encrypted
row 54 encrypted
row 55 encrypted
row 56 encrypted
row 57 encrypted
row 58 encrypted
row 59 encrypted
row 60

In [12]:
# Function to monitor CPU and RAM usage
def get_cpu_and_ram_utilization(pid, duration):
    process = psutil.Process(pid)
    cpu_usages = []
    ram_usages = []
    start_time = time.time()
    while time.time() - start_time < duration:
        # with monitor_lock:
        cpu_usage = process.cpu_percent(interval=1)
        ram_usage = process.memory_percent()
        cpu_usages.append(cpu_usage)
        ram_usages.append(ram_usage)
        print(f"CPU utilization: {cpu_usage}% | RAM utilization: {ram_usage}%")
    total_cpu_usage = sum(cpu_usages)
    return total_cpu_usage, cpu_usages, ram_usages

In [13]:
pid = os.getpid()

# Duration to monitor CPU usage
monitor_duration = 30300

time.sleep(15)

In [14]:
results={}
# Logistic Regression SGD training function
def logistic_regression_sgd(X, y, n, learning_rate=0.01, epochs=3):
    train_time_start = time.time()
    weights = np.zeros(n)

    weights_enc = [public_key.encrypt(i) for i in weights]
    bias = 0
    bias_enc = public_key.encrypt(bias)

    for epoch in range(epochs):
        count=0
        indices = np.random.permutation(m)
        for i in indices:
            xi = enc_X_train[i]
            yi = enc_y_train[i]

            linear_output = sum(paillier_multiplication_cloud(i, j) for i, j in zip(xi, weights_enc))
            linear_output = linear_output + bias_enc
            y_pred = sigmoid(linear_output)
            db = y_pred + (yi * (-1))
            dw = [paillier_multiplication_cloud(i, db) for i in xi]

            for i in range(len(weights_enc)):
                weights_enc[i] = weights_enc[i] + (dw[i] * (-1) * learning_rate)
            bias_enc = bias_enc + (db * -1) * learning_rate
            print(f"epoch={epoch},iter={count} executed")
            count=count+1

    train_time_end = time.time()
    train_time = train_time_end - train_time_start

    results['train_time'] = train_time
    results['weights'] = weights_enc
    results['bias'] = bias_enc
    print(results['train_time'])

In [ ]:
# Start the training thread
train_thread = Thread(target=logistic_regression_sgd, args=(enc_X_train, enc_y_train, n, learning_rate))
train_thread.start()

# Monitor CPU and RAM usage while training
total_cpu_usage, cpu_usages, ram_usages = get_cpu_and_ram_utilization(pid, monitor_duration)
train_thread.join()

print(f"Total CPU utilization over {monitor_duration} seconds: {total_cpu_usage}%")
print("CPU usage per second:", cpu_usages)
print("RAM usage per second:", ram_usages)

# Calculate total CPU resource consumption in 'CPU-seconds'
cpu_seconds = sum(cpu_usages) / 100
print(f"Total CPU resource consumption: {cpu_seconds} CPU-seconds")

Streaming output truncated to the last 5000 lines.
epoch=1,iter=271 executed
CPU utilization: 100.2% | RAM utilization: 1.7765224078022883%
CPU utilization: 104.8% | RAM utilization: 1.7765224078022883%
CPU utilization: 131.3% | RAM utilization: 1.7765224078022883%
CPU utilization: 100.0% | RAM utilization: 1.7765224078022883%
CPU utilization: 97.3% | RAM utilization: 1.7765224078022883%
CPU utilization: 96.5% | RAM utilization: 1.7765224078022883%
epoch=1,iter=272 executed
CPU utilization: 108.4% | RAM utilization: 1.7765224078022883%
CPU utilization: 100.1% | RAM utilization: 1.7765224078022883%
CPU utilization: 96.2% | RAM utilization: 1.7765224078022883%
CPU utilization: 100.7% | RAM utilization: 1.7765224078022883%
CPU utilization: 100.9% | RAM utilization: 1.7765224078022883%
epoch=1,iter=273 executed
CPU utilization: 100.3% | RAM utilization: 1.7765224078022883%
CPU utilization: 99.8% | RAM utilization: 1.7765224078022883%
CPU utilization: 101.9% | RAM utilization: 1.77652240780

In [ ]:
# Prepare encrypted test data
enc_X_test = [[public_key.encrypt(j) for j in i] for i in X_test]

weights_enc = results['weights']
bias_enc = results['bias']


In [ ]:
y_pred_enc = []
y_pred_unenc = []

test_time_start = time.time()

# Make predictions on test data
for i in enc_X_test:
    summ = sum(paillier_multiplication_cloud(p, q) for p, q in zip(i, weights_enc))
    summ = summ + bias_enc
    y_pred = sigmoid(summ)
    y_pred_enc.append(y_pred)

# Decrypt predictions
for i in y_pred_enc:
    d = private_key.decrypt(i)
    y_pred_unenc.append(1 if d >= 0.5 else 0)

test_time_end = time.time()
test_time = test_time_end - test_time_start
print(test_time)

In [ ]:
from sklearn.metrics import accuracy_score
accuracy = accuracy_score(y_test, y_pred_unenc)
print(f'Accuracy: {accuracy}')

In [ ]:
weights_unenc=[]
for i in weights_enc:
  p=private_key.decrypt(i)
  weights_unenc.append(p)

print(weights_unenc)

In [ ]:
bias_unenc=private_key.decrypt(bias_enc)
print(bias_unenc)

In [ ]:
import joblib
joblib.dump((weights_unenc, bias_unenc, accuracy, y_pred_unenc, cpu_seconds, cpu_usages, ram_usages, results['train_time'], test_time), 'variables_paillier.pkl')

In [ ]:
time.sleep(11600)